# Setup
Before starting with the tasks of the assignment, initialize spark.


In [1]:
import sys
from pathlib import Path

p = Path().resolve()
BASE_PATH =  p.parent if p.name == "src" else p

# To be sure that the src path is in PYTHONPATH
# TODO: Check if this requirement exists
sys.path.append(str(BASE_PATH / "src"))

import operator
import json
import numpy as np
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from pyspark import RDD, SparkContext
from pyspark.ml.feature import (
    IDF,
    ChiSqSelector,
    RegexTokenizer,
    HashingTF,
    StopWordsRemover,
    StringIndexer,
    VectorIndexer,
)

from exercise2.model.review import Review
from exercise2.split_text import split_text
from exercise2.task1.util import calculate_chi_squares, merge_dicts, printable_category, calculate_chi_square_per_token

LOCAL = True

if LOCAL:
    spark: SparkSession = SparkSession.builder \
        .appName("local") \
        .config("spark.driver.host", "localhost") \
        .config("spark.driver.bindAddress", "localhost") \
        .getOrCreate()
    sc: SparkContext = spark.sparkContext
else:
    spark: SparkSession = SparkSession.builder \
        .appName("cluster") \
        .config("spark.executor.instances", 435) \
        .getOrCreate()
    sc: SparkContext = spark.sparkContext
    sc.addPyFile(str(BASE_PATH / "src" / "exercise2.zip"))

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/21 17:11:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


PosixPath('/home/lukas/Programming/uni/data_intensive_computing/exercise2/src')

# Example 1: RDD
Redo the first assignment, this time utilizing RDDs. 

Start by loading the reviews dataset:

In [2]:
if LOCAL:
    reviews_location = BASE_PATH / "resource" / "reviews_devset.json" #_first1000.json" #"reviews_devset.json"
else:
    # reviews_location = "hdfs:///user/dic24_shared/amazon-reviews/full/reviews_devset.json"
    reviews_location = "hdfs:///user/dic24_shared/amazon-reviews/full/reviewscombined.json"
reviews: RDD = sc.textFile(str(reviews_location)).map(json.loads)
reviews_cnt = reviews.count()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/home/lukas/Programming/uni/data_intensive_computing/exercise2/src/resource/reviews_devset.json
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:304)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:244)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:332)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:208)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:294)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:290)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:294)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.

Once the reviews are available, count the documents per category.

In [ ]:
category_counts = reviews.map(lambda r: (r["category"], 1)) \
        .reduceByKey(operator.add) \
        .collectAsMap()

Split the reviews text into separate tokens, filter them using the stopwords (loaded from disk)
and reduce the produced counts into a map of maps with token on the top level, each token is 
assigned to a map, which contains the review counts for each category.

In [ ]:
with open(BASE_PATH / "resource" / "stopwords.txt", "r") as file:
    stopwords = set([line.strip() for line in file.readlines()])

# <term>: {<cat>: cnt, <cat>: cnt, ...}
category_counts_per_token = reviews \
        .flatMap(lambda r: ([(token, r["category"]) for token in set(split_text(r["reviewText"]))])) \
        .filter(lambda r: r[1] not in stopwords) \
        .mapValues(lambda category: {category: 1}) \
        .reduceByKey(lambda dict1, dict2: merge_dicts(dict1, dict2))

Now that we have the counts per token and category, we calculate the chi square values and sort
the values to filter for the top 75 tokens per category.

In [ ]:
top75_tokens_per_category: RDD = category_counts_per_token \
        .flatMap(lambda cur_category_counts: calculate_chi_square_per_token(cur_category_counts, category_counts, reviews_cnt)) \
        .groupByKey().mapValues(list) \
        .mapValues(lambda val: sorted(val, key=lambda val: val[1])[:-75:-1]) \
        .sortByKey()

Prepare the job result by concenating all tokens to a list of all top tokens 
and convert the lists to strings for printing.

In [ ]:
top75_tokens_str = "\n".join(
        top75_tokens_per_category.map(lambda el: printable_category(el[0], el[1])).collect())

top_tokens: list[str] = top75_tokens_per_category \
        .flatMap(lambda el: [tup[0] for tup in el[1]]) \
        .distinct() \
        .sortBy(lambda el: el).collect()
top_tokens_str = " ".join(top_tokens)

result = top_tokens_str + '\n' + top75_tokens_str
result

Write the result to a file.

In [ ]:
with open(BASE_PATH / "output_rdd.txt", "w") as file:
    file.writelines(result)

# Example 2: DataFrames: Spark ML & Pipelines


In [ ]:
reviews_df = spark.read.json(str(reviews_location))
reviews_df.head()

In [ ]:
tokenizer = RegexTokenizer(
    minTokenLength=2,
    pattern="[\s\d\(\)\[\]{}\.!\?,;:\+=\-_\"'`~#@&\*%€\$§\\\/]",
    inputCol="reviewText",
    outputCol="tokens",
)
stopwords_remover = StopWordsRemover(
    inputCol=tokenizer.getOutputCol(), outputCol="stopWordsFiltered"
)
indexer = StringIndexer(
    inputCol="category",
    outputCol="indexedCategory"
)
hashingTF = HashingTF(inputCol=stopwords_remover.getOutputCol(), outputCol="TF")
idf = IDF(inputCol=hashingTF.getOutputCol(), outputCol="TFIDF")
chiSqSelector = ChiSqSelector(
    numTopFeatures=2000,
    featuresCol=idf.getOutputCol(),
    labelCol=indexer.getOutputCol(),
    outputCol="chiSq",
)

In [ ]:
pipeline = Pipeline(
    stages=[
        tokenizer,
        stopwords_remover,
        indexer,
        hashingTF,
        idf,
        chiSqSelector
    ]
)

model = pipeline.fit(reviews_df)
tokenized = model.transform(reviews_df)

tokenized.head(n=10)